In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, random_split, DataLoader
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.init as init
import itertools
from torchvision.utils import make_grid

batch_size=3
num_workers=2
lr = 0.0005
b1 = 0.5
b2 = 0.996
epochs=100

Dataset

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, data_dir, size=(256, 256), normalize=True):
        super().__init__()
        self.monet_dir = os.path.join(data_dir, 'monet_jpg')
        self.photo_dir = os.path.join(data_dir, 'photo_jpg')
        if normalize:
            self.transform = transforms.Compose([
                transforms.Resize(size),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize(size),
                transforms.ToTensor()                               
            ])
        self.monet_file = [os.path.join(self.monet_dir, name) for name in sorted(os.listdir(self.monet_dir))]
        self.photo_file = [os.path.join(self.photo_dir, name) for name in sorted(os.listdir(self.photo_dir))]

    def __getitem__(self, idx):
        rand_idx = int(np.random.uniform(0, len(self.monet_file)))
        photo_img = Image.open(self.photo_file[rand_idx])
        photo_img = self.transform(photo_img)
        monet_img = Image.open(self.monet_file[idx])
        monet_img = self.transform(monet_img)
        return monet_img,photo_img

    def __len__(self):
        return len(self.monet_file)

img_Data = ImageDataset('/kaggle/input/gan-getting-started')
img_Dataset = DataLoader(img_Data, batch_size=batch_size, num_workers = num_workers)

In [ ]:
def unnorm(img, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]):
    for t, m, s in zip(img, mean, std):
        t.mul_(s).add_(s)
        
    return img

Model

In [ ]:
def init_weights(net, init_type='normal', gain=0.02):
    def init_func(m):
        classname = m.__class__.__name__
        if hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
            init.normal_(m.weight.data, 0.0, gain)
            if hasattr(m, 'bias') and m.bias is not None:
                init.constant_(m.bias.data, 0.0)
        elif classname.find('BatchNorm2d') != -1:
            init.normal_(m.weight.data, 1.0, gain)
            init.constant_(m.bias.data, 0.0)
    net.apply(init_func)

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.ReflectionPad2d(1), # padding, keep the image size constant after next conv2d
            nn.Conv2d(in_channels, in_channels, 3),
            nn.InstanceNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_channels, in_channels, 3),
            nn.InstanceNorm2d(in_channels)
        )
    
    def forward(self, x):
        return x + self.block(x)

In [ ]:
class Generator(nn.Module):
    def __init__(self, input_channels, num_residual_blocks=6):
        super(Generator, self).__init__()

        # Initial convolution block
        output_channels=64
        model = [   nn.ReflectionPad2d(input_channels),
                    nn.Conv2d(input_channels, output_channels, 2*input_channels+1),
                    nn.InstanceNorm2d(output_channels),
                    nn.ReLU(inplace=True) ]

        # Downsampling
        in_features = output_channels
        out_features = in_features*2
        for _ in range(2):
            model += [  nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                        nn.InstanceNorm2d(out_features),
                        nn.ReLU(inplace=True) ]
            in_features = out_features
            out_features = in_features*2

        # Residual blocks
        for _ in range(num_residual_blocks):
            model += [ResidualBlock(in_features)]

        # Upsampling
        out_features = in_features//2
        for _ in range(2):
            model += [nn.ConvTranspose2d(in_features, out_features, 3, stride=2, padding=1, output_padding=1),
                      nn.InstanceNorm2d(out_features),
                      nn.ReLU(inplace=True) ]
            in_features = out_features
            out_features = in_features//2

        # Output layer
        model += [nn.ReflectionPad2d(input_channels),
                  nn.Conv2d(output_channels, input_channels, 2*input_channels+1),
                  nn.Tanh() ]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, input_nc):
        super(Discriminator, self).__init__()

        # A bunch of convolutions one after another
        model = [nn.Conv2d(input_nc, 64, 4, stride=2, padding=1),
                 nn.LeakyReLU(0.2, inplace=True) ]

        model += [nn.Conv2d(64, 128, 4, stride=2, padding=1),
                  nn.InstanceNorm2d(128), 
                  nn.LeakyReLU(0.2, inplace=True) ]

        model += [nn.Conv2d(128, 256, 4, stride=2, padding=1),
                  nn.InstanceNorm2d(256), 
                  nn.LeakyReLU(0.2, inplace=True) ]

        model += [nn.Conv2d(256, 512, 4, padding=1),
                  nn.InstanceNorm2d(512), 
                  nn.LeakyReLU(0.2, inplace=True) ]

        # FCN classification layer
        model += [nn.Conv2d(512, 1, 4, padding=1)]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        x =  self.model(x)
        return x 

In [ ]:
def update_req_grad(models, requires_grad=True):
    for model in models:
        for param in model.parameters():
            param.requires_grad = requires_grad

In [ ]:
class CycleGAN(object):
    def __init__(self, in_channels, out_channels, epochs, cuda_device,lr,b1,b2,decay_epoch=0, lambda_id=5.0, lambda_cycle=10.0):
        
        self.device = cuda_device
        self.epochs = epochs
        self.lambda_id = lambda_id
        self.lambda_cycle = lambda_cycle
        self.G_AB = Generator(in_channels)
        self.F_BA = Generator(in_channels)
        self.D_A = Discriminator(in_channels)
        self.D_B = Discriminator(in_channels)
        init_weights(self.G_AB)
        init_weights(self.F_BA)
        init_weights(self.D_A)
        init_weights(self.D_B)
        self.G_AB = self.G_AB.to(self.device)
        self.F_BA = self.F_BA.to(self.device)
        self.D_A = self.D_A.to(self.device)
        self.D_B = self.D_B.to(self.device)
        
        self.criterion_GAN = nn.MSELoss().cuda()
        self.criterion_cycle = nn.L1Loss().cuda()
        self.criterion_identity = nn.L1Loss().cuda()
        self.optimizer_G = torch.optim.Adam(itertools.chain(self.G_AB.parameters(), self.F_BA.parameters()), lr=lr, betas=(b1, b2))
        self.optimizer_D_A = torch.optim.Adam(self.D_A.parameters(), lr=lr, betas=(b1, b2))
        self.optimizer_D_B = torch.optim.Adam(self.D_B.parameters(), lr=lr, betas=(b1, b2))

        lambda_func = lambda epoch: 1 - max(0, epoch-decay_epoch)/(epochs-decay_epoch)
        self.lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(self.optimizer_G, lr_lambda=lambda_func)
        self.lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(self.optimizer_D_A, lr_lambda=lambda_func)
        self.lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(self.optimizer_D_B, lr_lambda=lambda_func)
        
    def sample_images(self,real_A, real_B, figside=1.5):
        assert real_A.size() == real_B.size(), 'The image size for two domains must be the same'
        cuda = torch.cuda.is_available()
        Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor
        self.G_AB.eval()
        self.F_BA.eval()        
        real_A = real_A.type(Tensor)
        fake_B = self.G_AB(real_A).detach()
        real_B = real_B.type(Tensor)
        fake_A = self.F_BA(real_B).detach()
        
        nrows = real_A.size(0)
        real_A = make_grid(real_A, nrow=nrows, normalize=True)
        fake_B = make_grid(fake_B, nrow=nrows, normalize=True)
        real_B = make_grid(real_B, nrow=nrows, normalize=True)
        fake_A = make_grid(fake_A, nrow=nrows, normalize=True)
    
        image_grid = torch.cat((real_A, fake_B, real_B, fake_A), 1).cpu().permute(1, 2, 0)
    
        plt.figure(figsize=(figside*nrows, figside*4))
        plt.imshow(image_grid)
        plt.axis('off')
        plt.show() 
        
    def train(self, data):
        for epoch in range(self.epochs):
            for i, (A_real, B_real) in enumerate(data):
                A_real, B_real = A_real.to(self.device), B_real.to(self.device)
                update_req_grad([self.D_A, self.D_B], False)
                self.optimizer_G.zero_grad()
                self.G_AB.train()
                self.F_BA.train() 
        
        
                # Forward pass through generator
                B_fake = self.G_AB(A_real)
                A_fake = self.F_BA(B_real)
                 
                A_cycle = self.F_BA(B_fake)
                B_cycle = self.G_AB(A_fake)

                B_id = self.G_AB(B_real)
                A_id = self.F_BA(A_real)
                
                # generator losses - identity, Adversarial, cycle consistency
                id_loss = (self.criterion_identity(A_id, A_real) + self.criterion_identity(B_id, B_real)) * self.lambda_id 
                cycle_loss = (self.criterion_cycle(A_cycle, A_real) + self.criterion_cycle(B_cycle, B_real)) * self.lambda_cycle

                A_GAN = self.D_A(A_fake)
                B_GAN = self.D_B(B_fake)
                valid = torch.ones(A_GAN.size()).to(self.device)
                G_loss = self.criterion_GAN(A_GAN, valid) + self.criterion_GAN(B_GAN, valid)
                    
                total_loss = G_loss + id_loss + cycle_loss
                    
                # backward pass
                total_loss.backward()
                self.optimizer_G.step()
                    
                # Train Discriminator A
                update_req_grad([self.D_A, self.D_B], True)
                fake = torch.zeros(A_GAN.size()).to(self.device)
                self.optimizer_D_A.zero_grad()
        
                loss_real = self.criterion_GAN(self.D_A(A_real), valid)
                loss_fake = self.criterion_GAN(self.D_A(A_fake.detach()), fake)
                loss_D_A = (loss_real + loss_fake) / 2
        
                loss_D_A.backward()
                self.optimizer_D_A.step()
        
                # Train Discriminator B
                self.optimizer_D_B.zero_grad()
        
                loss_real = self.criterion_GAN(self.D_B(B_real), valid)
                loss_fake = self.criterion_GAN(self.D_B(B_fake.detach()), fake)
                loss_D_B = (loss_real + loss_fake) / 2
                                               
                loss_D_B.backward()
                self.optimizer_D_B.step()
    
            self.lr_scheduler_G.step()
            self.lr_scheduler_D_A.step()
            self.lr_scheduler_D_B.step()
                    
            if (epoch+1) % 10 == 0:
                img_monet, img_photo = next(iter(data))
                self.sample_images(img_monet, img_photo)
                    
                loss_D = (loss_D_A + loss_D_B) / 2
                print(f'[Epoch {epoch+1}/{self.epochs}]')
                print(f'[G loss: {total_loss.item()} | identity: {id_loss.item()} GAN: {G_loss.item()} cycle: {cycle_loss.item()}]')
                print(f'[D loss: {loss_D.item()} | D_A: {loss_D_A.item()} D_B: {loss_D_B.item()}]')       

In [ ]:
cuda_device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(cuda_device)
GAN=CycleGAN(3, 3, epochs, cuda_device, lr, b1, b2)

In [ ]:
GAN.train(img_Dataset)

Test

In [ ]:
class PhotoDataset(Dataset):
    def __init__(self, data_dir, size=(256, 256), normalize=True):
        super().__init__()
        self.photo_dir = os.path.join(data_dir, 'photo_jpg')
        if normalize:
            self.transform = transforms.Compose([
                transforms.Resize(size),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize(size),
                transforms.ToTensor()                               
            ])
        self.photo_file = [os.path.join(self.photo_dir, name) for name in sorted(os.listdir(self.photo_dir))]

    def __getitem__(self, idx):
        photo_img = Image.open(self.photo_file[idx])
        photo_img = self.transform(photo_img)
        return photo_img

    def __len__(self):
        return len(self.photo_file)

photo_Data = PhotoDataset('/kaggle/input/gan-getting-started')
photo_Dataset = DataLoader(photo_Data, batch_size=batch_size, num_workers = num_workers)

In [ ]:
!mkdir ../images

In [ ]:
GAN.F_BA.eval()
trans = transforms.ToPILImage()
for i, photo in enumerate(photo_Dataset):
    with torch.no_grad():
        pred_monet = GAN.F_BA(photo.to(cuda_device)).cpu().detach()
    pred_monet = unnorm(pred_monet)
    for j in range(batch_size):
        img = trans(pred_monet[j]).convert("RGB")
        img.save("../images/" + str(3*i+j+1) + ".jpg")

In [ ]:
import shutil
shutil.make_archive("/kaggle/working/images", 'zip', "/kaggle/images")